# First steps

In [ ]:
import sisl
# We define the root directory where our files are
siesta_files = sisl._environ.get_environ_variable("SISL_FILES_TESTS") / "sisl" / "io" / "siesta"

## Activating the viz framework

The first thing you will need to do in order to use plots is to import `sisl.viz`:

In [ ]:
import sisl.viz
from sisl.viz import Plot

This will load the appropiate things into sisl to use the visualization tools. You can also control the loading of the framework with an environment variable:

```
SISL_VIZ_AUTOLOAD=True
```

will load the framework on `import sisl`, so you won't need to explicitly import it.

<div class="alert alert-info">
    
Note
    
If you use sisl to run high performance calculations where you initialize sisl frequently it's better to have the autoloading turned off (default), as it might introduce an overhead of about a second.
    
</div>

Now that the framework has been loaded, we can start plotting!

## My first plots

The most straightforward way to plot things in sisl is to call the `Plot` class, which you can import as shown in the next cell:

In [ ]:
from sisl.viz import Plot

Plot(siesta_files / "SrTiO3.bands")

Note how we just passed the path to our bands file and **sisl recognized what was the plot that we wanted to generate**.

Let's try now passing a *.RHO* file to check the electronic density:

In [ ]:
Plot(siesta_files / "SrTiO3.RHO", axes="xy", nsc=[2,1,1], zsmooth='best')

You probably noticed that we used some extra arguments (`axes`, `nsc`, `zsmooth`) to get the exact plot that we wanted. These arguments are called **settings**. Settings define how the plot will process and show your plot. You can **provide settings on initialization or update them later**. 

`Plot()` returns a plot object. If you want to keep that plot object for later (to do any modification on it) you will have to, of course, store it in a variable. Let's do that:

In [ ]:
rho_plot = Plot(siesta_files / "SrTiO3.RHO", axes="xy", nsc=[2,1,1], zsmooth="best")

And now that we have it, let's try to get some help from it to understand the plot object better.

In [ ]:
print(rho_plot.__class__)
print(rho_plot.__doc__)

We can see two interesting things:

- Our plot is a `GridPlot`, not simply a `Plot`. This means that **it knows you are dealing with a grid** and consequently it will help you by **providing useful methods and settings**.
- On the documentation, under `Parameters`, you can see the arguments that this plot understands. If you've guessed these are the so-called *settings*, then you've guessed right! A way to know the current settings of your plot is to check the `settings` attribute:


In [ ]:
rho_plot.settings

The names might already give you a quick intuition of what each setting does, but for more detail you can go to the documentation. The **showcase notebooks** show examples and are designed to **help you understand what each setting does in a visual way**. It is always worth checking them out if you are dealing with a new plot type!

One of the interesting methods that grid plots have is the `scan` method. Here we use it to do a simple scan of 15 steps with the default settings, but you can play with it:

In [ ]:
rho_plot.scan(along="z", num=15)

## Plotable objects

In this section we'd like to point out that using the `Plot` class is not the most convenient thing for day to day usage. Instead, everything that is *plotable* in `sisl` will receive a `plot` method that you can use. One example of a plotable object is the `bandsSileSiesta`:

In [ ]:
bands_sile = sisl.get_sile(siesta_files/"SrTiO3.bands")
bands_sile.plot()

Although the `plot` attribute is not exactly a method, but **a manager that organizes all the plotting possibilities for an object**. If you call it, as we did, you get the default plot, but you can specify which plot type you want specifically:

In [ ]:
bands_sile.plot.bands(bands_color="red")

One can quickly check what are the options:

In [ ]:
dir(bands_sile.plot)

And you then see that there's the option to plot the fatbands from this object. We won't do it here because it needs the `.WFSX` file, which we don't have.

## Updating your plots

As we mentioned earlier, **plots have settings and they can be updated**. This stems from the fact that the framework is designed with GUIs in mind, where the user will have visual input fields that they may tweak to see how the plot changes. So you might do as if you were interacting from a GUI and update the settings:

In [ ]:
rho_plot.update_settings(z_range=[1, 3], axes="xyz", isos=[{"frac": 0.05, "color":"lightgreen", "opacity": 0.3}])

The most important thing is that, by the time we do this update, the *.RHO* file could have changed its location or even disappeared and it wouldn't matter. When you update a setting, **the plot reruns only from the point where that setting is used**. This **avoids rerunning time-consuming initializations** like reading a very big file or diagonalizing a hamiltonian.

However, this is not the only useful point. Since **plots are self-contained**, you can **share this plot with someone else and they will be able to tweak all the settings** that they wish if they don't involve reading data again. Isn't this nice? This brings us to the next section. 

## Storing and loading plots

After a time-consuming calculation or data reading, you clearly want your results to be saved. Plots provide a `save` method:

In [ ]:
rho_plot.save("rho_plot.plot")

As discussed in the last paragraph of the previous section this **stores not only the current visualization**, but the full self contained plot that you can tweak as you wish when you load it again:

In [ ]:
rho_plot_from_colleague = sisl.viz.load("rho_plot.plot")

And do whatever you want with it:

In [ ]:
rho_plot_from_colleague.update_settings(axes="x")

In [ ]:
# Let's clean the working directory.
import os
os.remove("rho_plot.plot")

You might ask yourself now what happens if you just want to store the representation, not the full self-contained plot. For this, we first need to discuss the next section.

## Different plotting backends

Hidden between all the settings, you can find a **very special setting**: `backend`.

Initially, the visualization framework was written to plot things using `plotly`. However, we noticed that this might not be the appropiate choice for everyone. Therefore, we changed the design to make it very modular and **allow rendering the plot with any framework you like**. There's a dedicated notebook on how to register your own backends. Here however we just want to show you **how you can switch between the sisl-provided backends**. It is very simple:

In [ ]:
rho_plot_from_colleague.update_settings(backend="matplotlib", axes="x")

In [ ]:
rho_plot_from_colleague.update_settings(backend="plotly", axes="xy")

In [ ]:
rho_plot_from_colleague.update_settings(backend="matplotlib")

Note that you can always know what backends are available for the plot by checking the options of the `backend` parameter:

In [ ]:
rho_plot_from_colleague.get_param("backend").options

Note that the options here will only show the backends that have been loaded. `sisl` only **loads backends if the required python packages are present**. Currently, `sisl` provides backends for three frameworks: `plotly`, `matplotlib` and `blender`.

If you have one backend selected, you will have available all the methods that the framework provides. For example, if you are using the `matplotlib` backend, you can use all the methods that matplotlib implements for the `Axes` object directly on the plot. You also have the figure (axes) under the `figure` (`axes`) attribute, for whatever you want to do.

Let's for example draw a line, using `Axes.plot`:

In [ ]:
rho_plot_from_colleague.plot([1,2,3,4], [0,0,1,2])
rho_plot_from_colleague

And now let's do the same with `plotly`. In this case, all methods are looked for in the `Figure` object that is stored under the `figure` attribute.

In [ ]:
rho_plot_from_colleague.update_settings(backend="plotly")
rho_plot_from_colleague.add_scatter(x=[1,2,3,4], y=[0,0,1,2])

At this point, you probably already know how you will be able to save these plots to images, html or whatever other format. Use **the methods that each framework provides**!

Also, this will also allow you to modify the plot as you wish (*adding lines, changing titles, showing legends...*) once `sisl` has render it. Again, you just have to use the methods that the framework provides to do so :)

## Discover more

Until here, we have covered the most basic concepts of the framework. If you enjoyed it, we encourage you to check the rest of notebooks to find out about more specific and complex aspects of it.

_______________
This next cell is just to create the thumbnail for the notebook in the docs

In [ ]:
thumbnail_plot = rho_plot_from_colleague

if thumbnail_plot:
    thumbnail_plot.show("png")